In [1]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

SYSTEM_CONTENT = AMAZON_CONTENT_SYSTEM

# Constants for column names
USER_COLUMN_NAME = 'reviewerID'
TITLE_COLUMN_NAME = 'title'
ITEM_ID_COLUMN = 'asin'
RATING_COLUMN_NAME = 'rating'



In [2]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/large_merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/output/title_large_predictions_zero_shot.csv')
print(f'Zero shot save path: {ZERO_SHOT_SAVE_PATH}')

# few shot save path
FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/output/title_large_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')



# few shot save path
FEW_SHOT_1_OBS_RERUN_PATH = os.path.join(DATA_DIR, 'amazon-beauty/output/rerun_title_large_1_test_predictions_few_shot.csv')
print(f'Few shot rerun path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/data/amazon-beauty/large_merged_data.csv
Zero shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/data/amazon-beauty/output/title_large_predictions_zero_shot.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/data/amazon-beauty/output/title_large_1_test_predictions_few_shot.csv
Few shot rerun path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/data/amazon-beauty/output/title_large_1_test_predictions_few_shot.csv


# Data Overview

In [3]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7502 entries, 0 to 7501
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rating          7502 non-null   float64
 1   verified        7502 non-null   bool   
 2   reviewTime      7502 non-null   object 
 3   reviewerID      7502 non-null   object 
 4   asin            7502 non-null   object 
 5   reviewerName    7502 non-null   object 
 6   reviewText      7496 non-null   object 
 7   summary         7496 non-null   object 
 8   unixReviewTime  7502 non-null   object 
 9   vote            1133 non-null   object 
 10  style           4702 non-null   object 
 11  category        7502 non-null   object 
 12  tech1           2 non-null      object 
 13  description     7502 non-null   object 
 14  fit             0 non-null      float64
 15  title           7502 non-null   object 
 16  also_buy        7502 non-null   object 
 17  tech2           0 non-null      f

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,1.0,False,2015-08-25,A2RYSCZOPEXOCQ,9790787006,The Cat Next Door,"I use a lot of perfume, I go through a new bot...",This is not going to be my favorite scent.,2015-08-25,NaN,...,NaN,Jenna Jameson,[],298.0,"['B00357FTX8', 'B01NBID7FJ', 'B0017JT658']","{'Shipping Weight:': '12.8 ounces (', 'ASIN: '...",All Beauty,NaN,NaN,13.85
1,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
2,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50
3,4.0,True,2018-02-09,A3J0LOCVOIEDNE,B000050FDY,LouRevue,Works great and keeps my Braun shaver working/...,Four Stars,2018-02-09,NaN,...,NaN,Braun,[],1.0,"['B000KND8IS', 'B07L9PW8R3', 'B071KCNQTR', 'B0...",{},All Beauty,NaN,NaN,11.94
4,5.0,True,2018-01-14,AD1WVH9MP1ZS6,B000050FDY,SpecialK,"Cleans my shaver pretty well. A bit pricey, bu...",Five Stars,2018-01-14,NaN,...,NaN,Braun,[],1.0,"['B000KND8IS', 'B07L9PW8R3', 'B071KCNQTR', 'B0...",{},All Beauty,NaN,NaN,11.94


# Zero-shot (OpenAI API)

In [5]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_prediction=[TITLE_COLUMN_NAME],
                                       user_column_name=USER_COLUMN_NAME,
                                       title_column_name=TITLE_COLUMN_NAME,
                                       asin_column_name=ITEM_ID_COLUMN,
                                       rating_column_name=RATING_COLUMN_NAME,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=ZERO_SHOT_SAVE_PATH,
                                       system_content=SYSTEM_CONTENT)

Constructed Prompt for zero-shot approach:

The prompt:
**********
How will user rate this product title: Toni&amp;Guy Glamour Volume Plumping Whip, 2.82 Fluid Ounce? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.
**********


System Fingerprint: fp_cbe4fa03fe

API call response: "4"
Extracted rating: 4.0



----------------------------------------------------------------------------------
Constructed Prompt for zero-shot approach:

The prompt:
**********
How will user rate this product title: Philips Sonicare UV Sanitizer? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.
**********


System Fingerprint: fp_cbe4fa03fe

API call response: "4"
Extracted rating: 4.0



----------------------------------------------------------------------------------
Constructed Prompt for zero-shot approach:

The prompt:
**********
How will us

,user_id,item_id,title,actual_rating,predicted_rating
0,A10ZBR6O8S8OCY,B00EF1QRMU,"Toni&amp;Guy Glamour Volume Plumping Whip, 2.8...",4.0,4.0
1,ABPNZ9RKXOP0E,B000V5Z4J6,Philips Sonicare UV Sanitizer,5.0,4.0
2,A198RQKA8VFDQ8,B00AQXD2HO,SenzAway Tooth Desensitizing Gel,5.0,4.0
3,A6FIAB28IS79,B000H6A02A,Remington MS2-390 Microscreen Rechargeable/Cor...,5.0,4.0
4,A3FKGKUCI3DG9U,B00W259T7G,Pre de Provence Artisanal French Soap Bar Enri...,5.0,4.0
...,...,...,...,...,...
709,A152C9DCMDC21C,B01B7ADZEO,Skinfood Organic Coconut Oil Moisturizer,5.0,4.0
710,A2KYOC9YNAL3B6,B00W4VTPB8,"Tomato and Betonite Clay Soap Bar, Clear compl...",2.0,5.0
711,A9YTY73E5PTVA,B019ADT8T6,Zoella Beauty All That Glimmers - Fluffy Powde...,3.0,4.0
712,ARTA80PYMX8CN,B0199MCIK4,BORN PRETTY 9 Tips Laser Nail Vinyls Vintage D...,5.0,4.0


In [4]:
# Read the data
zeroshot_saved_data = pd.read_csv(ZERO_SHOT_SAVE_PATH)

# Display the original data types
print("Original Data Types:")
print(zeroshot_saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
zeroshot_saved_data['is_rating_float'] = pd.to_numeric(zeroshot_saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = zeroshot_saved_data[zeroshot_saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


Original Data Types:
user_id              object
item_id              object
title                object
actual_rating       float64
predicted_rating     object
dtype: object


Total number of rows with non-float ratings: 2
Rerun indices: [230, 234]
Rows with non-float ratings:


,user_id,item_id,title,actual_rating,predicted_rating,is_rating_float
230,A3A5ATNHC0QPQA,B001OHV1H4,Avalon Grapefruit and Geranium Smoothing Shamp...,5.0,"(None, ""Error communicating with OpenAI: ('Con...",False
234,A222BBVJ75BTFE,B001OHV1H4,Avalon Grapefruit and Geranium Smoothing Shamp...,4.0,"(None, ""Request timed out: HTTPSConnectionPool...",False


In [5]:
%%time

zero_shot_saved_data = pd.read_csv(ZERO_SHOT_SAVE_PATH)

# Rerun predictions for failed cases and save the updated data
rerun_save_path = os.path.join(DATA_DIR, 'movie-ml-latest-small/output/rerun_title_large_predictions_zero_shot.csv')
columns_for_prediction = ['title']
updated_data = rerun_failed_zero_shot_predictions(zero_shot_saved_data, ZERO_SHOT_SAVE_PATH, rerun_save_path, columns_for_prediction, PAUSE_EVERY_N_USERS, SLEEP_TIME)

# Remove rows with non-float ratings and save the cleaned data
cleaned_data = updated_data[pd.to_numeric(updated_data['predicted_rating'], errors='coerce').notna()]
cleaned_data.to_csv(ZERO_SHOT_SAVE_PATH, index=False)

# Evaluate the model predictions
evaluate_model_predictions_rmse_mae(ZERO_SHOT_SAVE_PATH, NUM_EXAMPLES, 'actual_rating', 'predicted_rating')


Re-running predictions for 2 failed cases.
Predictions saved to /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/data/movie-ml-latest-small/output/rerun_title_large_predictions_zero_shot.csv
RMSE: 1.0850 (95% CI: (1.0366, 1.1367)) ± 0.0005
MAE: 0.9536 (95% CI: (0.9157, 0.9930)) ± 0.0004

First few actual vs predicted ratings:
Actual: 4.0, Predicted: 4.0000
Actual: 5.0, Predicted: 4.0000
Actual: 5.0, Predicted: 4.0000
Actual: 5.0, Predicted: 4.0000
Actual: 5.0, Predicted: 4.0000
CPU times: user 3.34 s, sys: 7.68 ms, total: 3.35 s
Wall time: 3.37 s


(1.0850339055944214, 0.9536310393258427)

# Few-shot (OpenAI API)


+ For each user, we'll use 4 of their ratings as training data to predict ratings for the rest of their products. Finally, we'll evaluate the predictions against the actual ratings to calculate the overall RMSE and MAE.

+ The rating_history_str now includes both the title and the review text for each of the training data rows

# 1 observation per reviewer - Few-shot OpenAI

In [5]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=[TITLE_COLUMN_NAME],
                                       columns_for_prediction=[TITLE_COLUMN_NAME],
                                       title_column_name=TITLE_COLUMN_NAME, 
                                       user_column_name=USER_COLUMN_NAME,
                                       asin_column_name=ITEM_ID_COLUMN,
                                       rating_column_name=RATING_COLUMN_NAME,
                                       obs_per_user=TEST_OBSERVATION_PER_USER,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=FEW_SHOT_1_OBS_SAVE_PATH,
                                       system_content=SYSTEM_CONTENT)


Total number of users: 1608
Predicting rating for: "title: Toni&amp;Guy Glamour Volume Plumping Whip, 2.82 Fluid Ounce"
Rating history:
* title: Braun Clean &amp; Renew Refill Cartridges CCR - 2 Count (Packaging May Vary) - Rating: 5.0 stars
* title: Pre de Provence Maison French Dried Lavender Blossoms for Fragrance - Rating: 4.0 stars
* title: Philips Norelco QC5055 Power Hair Clipper - Rating: 4.0 stars
* title: Fekkai Full Blown Aerosol Foam Cond Us 6.6 Oz, 6.660-Fluid Ounce - Rating: 4.0 stars
Constructed Prompt for few-shot approach:

The prompt:
**********


Here is user rating history:
* title: Braun Clean &amp; Renew Refill Cartridges CCR - 2 Count (Packaging May Vary) - Rating: 5.0 stars
* title: Pre de Provence Maison French Dried Lavender Blossoms for Fragrance - Rating: 4.0 stars
* title: Philips Norelco QC5055 Power Hair Clipper - Rating: 4.0 stars
* title: Fekkai Full Blown Aerosol Foam Cond Us 6.6 Oz, 6.660-Fluid Ounce - Rating: 4.0 stars

Based on above rating history,

KeyboardInterrupt: 

In [ ]:
# Read the data
fewshot_saved_data = pd.read_csv(FEW_SHOT_1_OBS_SAVE_PATH)

# Display the original data types
print("Original Data Types:")
print(fewshot_saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
fewshot_saved_data['is_rating_float'] = pd.to_numeric(fewshot_saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = fewshot_saved_data[fewshot_saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
%%time 

rerun_failed_few_shot_predictions(data, 
                                  columns_for_training=[TITLE_COLUMN_NAME],
                                  columns_for_prediction=[TITLE_COLUMN_NAME],
                                  user_column_name=USER_COLUMN_NAME,
                                  title_column_name=TITLE_COLUMN_NAME,
                                  asin_column_name=ITEM_ID_COLUMN,
                                  rating_column_name=RATING_COLUMN_NAME,
                                  obs_per_user=TEST_OBSERVATION_PER_USER,
                                  pause_every_n_users=PAUSE_EVERY_N_USERS,
                                  sleep_time=SLEEP_TIME,
                                  save_path=FEW_SHOT_1_OBS_SAVE_PATH, 
                                  new_path=FEW_SHOT_1_OBS_RERUN_PATH,
                                  rerun_indices=rerun_indices,
                                  system_content=SYSTEM_CONTENT)

In [ ]:
# Read the data
fewshot_saved_data = pd.read_csv(FEW_SHOT_1_OBS_RERUN_PATH)

# Display the original data types
print("Original Data Types:")
print(fewshot_saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
fewshot_saved_data['is_rating_float'] = pd.to_numeric(fewshot_saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = fewshot_saved_data[fewshot_saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
evaluate_model_predictions_rmse_mae(
    data_path=FEW_SHOT_1_OBS_RERUN_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

# Limitations:

The model might not fully understand the nuanced relationships between products based on titles alone. Additional context or features might be needed for more accurate predictions.
This approach might be computationally expensive and slower than traditional matrix factorization or deep learning-based recommendation models, especially for a large number of users.

# References

+ https://platform.openai.com/docs/api-reference/authentication